### Preface 

In class I talked a bit about  maping out steps to executing projects. This would look like this: identify question and dataset(s) that may answer the question; import data; manipulate data; and then try and answer the question. The question part is hard, but this is more conceptual, not coding. The manipulation part is where coding skills are helpfull. Specifically, cleaning, merging, shaping the data to that the data set is useable to answer the question at hand. 

### Cleaning and String Methods on Dataframes

This notebook works through some cleaning examples that will probably help you in your project. Here we describe features of Pandas that allow us to clean data that, for reasons beyond our control, comes in a form that's not immediately amendable to analysis. This is the first of several such notebooks.

#### The Question (or want)...

We need to know what we're trying to do---what we want the data to look like. To borrow a phrase from our friend Tom Sargent, we say that we apply the want operator. Some problems we've run across that ask to be solved:

- We have too much data, would prefer to choose a subset.
- Row and column labels are contaminated.
- Numerical data is contaminated by commas (marking thousands); dollar signs; other non-numerical values, etc.
- Missing values are marked erratically.

What we want in each case is the opposite of what we have: we want nicely formatted numbers, clean row and column labels, and so on. We'll solve the first four problems here, the last one in the next notebook.

In [1]:
import pandas as pd                    # data package
import matplotlib.pyplot as plt        # graphics module  
import datetime as dt                  # date and time module
import numpy as np                     # foundation for pandas 

### Example: Chipotle data

This data comes from a New York Times story about the number of calories in a typical order at Chipotle. The topic doesn't particularly excite us, but the data raises a number of issues that come up repeatedly. We adapt some code written by Daniel Forsyth.

In [2]:
#url = 'https://raw.githubusercontent.com/TheUpshot/chipotle/master/orders.tsv'
path = "C://data_bootcamp//Data_Bootcamp_Fall_2017//data_bootcamp_1106//orders_dirty.csv"
# Double forward slashes for windows machines.

chp = pd.read_csv(path)  

print("Variable dtypes:\n", chp.dtypes, sep='')
# Lets checkout the datatypes that we have... are they what you expect?

chp.head()
#chp.tail()
#chp.shape

Variable dtypes:
order store id 1        object
quantity 2               int64
item name 3             object
choice description 4    object
item price 5            object
dtype: object


,order store id 1,quantity 2,item name 3,choice description 4,item price 5
0,1 Bucks County,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1 Bucks County,1,Izze,[Clementine],$3.39
2,1 Bucks County,1,Nantucket Nectar,[Apple],$3.39
3,1 Bucks County,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2 Bucks County,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


#### Issue #1: We have too much data, want to work with a subset.

Ok, so this is not really an issue here. This is about 5000 rows, length is small too. Not huge. But lets imagine that it was huge and we don't want deal with continually manipulating a big data set. We already know how to do this...we just use the `nrows` command when we read in the dataset.

In [3]:
chp = pd.read_csv(path, nrows = 500)   

print("Variable dtypes:\n", chp.dtypes, sep='')
# Lets checkout the datatypes that we have... are they what you expect?

chp.head()

chp.tail()

chp.shape

Variable dtypes:
order store id 1        object
quantity 2               int64
item name 3             object
choice description 4    object
item price 5            object
dtype: object


(500, 5)

Now the shape indicates that we only have 500 rows. Just as we specified. This was easy. 

One strategy is to write and test your code on only a subset of the data. Again the upside is that the code may run faster, its easier too look at and analyze. Then once you have everything sorted out, you simply change the code above and scale it up.

**Here is the issue to be mindfull of: the subset may not be "representative" of the entire data set.** For example, there may be issues in say row 1458 (e.g. missing values, different data types), that will only arise when the full data set is imported. This is just something to be mindfull of when pursuing this approach.

#### Issue #2: Row and column labels are contaminated.

Return to the head and the dypes and look at the varible names...

In [4]:
chp = pd.read_csv(path, nrows = 500)  

print("Variable dtypes:\n", chp.dtypes, sep='')
# Lets checkout the datatypes that we have... are they what you expect?

chp.head()

Variable dtypes:
order store id 1        object
quantity 2               int64
item name 3             object
choice description 4    object
item price 5            object
dtype: object


,order store id 1,quantity 2,item name 3,choice description 4,item price 5
0,1 Bucks County,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1 Bucks County,1,Izze,[Clementine],$3.39
2,1 Bucks County,1,Nantucket Nectar,[Apple],$3.39
3,1 Bucks County,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2 Bucks County,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


Here we see several issues that may slow us down, if fixed could help things.

- Notice how the variable names are sperated and then they have these numerical values in them (as if the person constructing the data wanted to help us by telling us the column number). We could simply slice the data set accordingly, or we could change the column names in a simplier way. Lets follow the later approach.

- Second, notice that the "order store id 1" value gives us a order number (note how one order has several entries) and then store id. This dataset has only one store and thus we may want to get rid of that, this will leave us with a numerical value which will index each oder that we could exploit.

Lets solve each of these issues inturn.

In [5]:
# One way to fix the names is just to rename them by hand like this...

#new_name_list = ["order_id", "quantity", "item_name", "choice_desc", "item_price"]

#chp.columns = new_name_list

In [6]:
# Another way is to use string methods on the column names and create something more usable.
# Here is a test run, what does this do?

test = "order store id 1"

test.rsplit(maxsplit=1)[0].replace(" ","_")

# So this splits the string into a list. The max split doess...
# Then the bracket says, take the first entry.
# Then the next part says replace the space with an underscore,
# this will help us call a column name more easily.

# What if we did not have max split?


'order_store_id'

In [7]:
# Now lets fix this all up for the data from
new_name_list = []

for var in chp.columns:
    new_name_list.append(var.rsplit(maxsplit=1)[0].replace(" ","_"))
    
# Then rename everything...

chp.columns = new_name_list

chp.head()    

,order_store_id,quantity,item_name,choice_description,item_price
0,1 Bucks County,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1 Bucks County,1,Izze,[Clementine],$3.39
2,1 Bucks County,1,Nantucket Nectar,[Apple],$3.39
3,1 Bucks County,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2 Bucks County,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


Great work!

So this fixed some issues with the columns, lets use the same idea to fix the issue with the order stor id, so get the "Bucks County" out of there.

In [8]:
# Again, lets test this out...

# Step one, pull off the number...

test = "1 Bucks County"
test2 = test.rsplit()[0] # same idea, don't use the max split option....

print(type(test2)) # I want this numerical, but its not...

# Step two, convert to floating point...

test2 = float(test2)
print(type(test2))

<class 'str'>
<class 'float'>


This gaves us a general appraoch to fixing the the order numbers. Here is the one trick, some of these operations have a bit different syntax since we will be operating on a dataframe object, not string object. To perform this operation, we need to use **Pandas string methods.** We can do the same thing to all the observations of a variable with so-called string methods. We append .str to a variable in a dataframe and then apply the string method of our choice. If this is part of converting a number-like entry that has mistakenly been given dtype object, we then convert its dtype with the astype method.

In [9]:
chp.head()
chp.columns
chp.order_store_id.head()

# Just to verify we are doing what we think we are...

chp.order_store_id = chp.order_store_id.str.rsplit().str[0].astype(int)
# Note that we need two str's here: one to do the split, the other to extract the first element.
# Then the last part of the code `astype` converts it to a string...


# note nothing changes unless we reassign everything. 

#

In [10]:
chp.head(20)

,order_store_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [11]:
print("Variable dtypes:\n", chp.dtypes, sep='')

Variable dtypes:
order_store_id         int32
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object


#### Issue #3: Numerical data is contaminated by commas (marking thousands); dollar signs; other non-numerical values, etc.

Ok, so we sorted out some issue with labels on the rows and columns. We still have the following issue that the item price is not a numerical value. Check above, the type of `item_price` is an object, not a float. If we want to do some kind of numerical calulation on this, then we need to convert it.

**Why is `item_price` not a numerical value?** ITs those dam dollar sighns. Someone put them their thinking they were being helpful, but it is giving us a headache. 

**How do we fix it?** Dude, in a very simmilar way above.

In [12]:
chp.item_price.head()

chp.item_price.str.replace("$","").head()

# So notice how this got the dollar sign out of there. We still have one more issue.
# That the type is still not a floating point value, so lets convert it.

#chp.item_price = chp.item_price.str.replace("$","").astype(float)

## Why is this not working??????

0     2.39 
1     3.39 
2     3.39 
3     2.39 
4    16.98 
Name: item_price, dtype: object

In [13]:
chp.item_price.sort_values(ascending=False).head(50)
# Lets order stuff to see what is going on, 
# The error above is saying there is something
# in the dataset called gift card...

453    gift card
454    gift card
448    gift card
449    gift card
450    gift card
451    gift card
452    gift card
455    gift card
405       $9.25 
237       $9.25 
207       $9.25 
184       $9.25 
181       $9.25 
219       $9.25 
222       $9.25 
444       $9.25 
236       $9.25 
239       $9.25 
127       $9.25 
421       $9.25 
390       $9.25 
373       $9.25 
342       $9.25 
335       $9.25 
418       $9.25 
420       $9.25 
293       $9.25 
439       $9.25 
286       $9.25 
285       $9.25 
266       $9.25 
255       $9.25 
254       $9.25 
243       $9.25 
145       $9.25 
499       $9.25 
56        $9.25 
95        $9.25 
121       $9.25 
61        $9.25 
481       $9.25 
494       $9.25 
478       $9.25 
84        $9.25 
88        $9.25 
92        $9.25 
81        $9.25 
108       $9.25 
9         $9.25 
8         $9.25 
Name: item_price, dtype: object

Ok, so what I'm going to show you is a different method to replace whole entries and assign them an missing value. (This will set us up for the next issue. 

In [14]:
chp.item_price = chp.item_price.str.replace("$","")
# Fix the dollar sign stuff first...

chp.item_price = chp.item_price.replace(to_replace=["gift card"], value=[np.nan])
# So lets walk through what this does, it takes the column, then uses the replace 
# comand, to_replace = ["what we want to replace"], then the value
# that we want to replace it with. We are goning to use the numpy NaN value
# which the dataframe will proplerly recognice as not a number.

# Note this could be a huge pain if there were differing random 
# strings floating around.

chp.item_price.sort_values(ascending=True).tail(20)


184    9.25 
181    9.25 
145    9.25 
127    9.25 
285    9.25 
121    9.25 
286    9.25 
293    9.25 
108    9.25 
95     9.25 
405    9.25 
499    9.25 
448      NaN
449      NaN
450      NaN
451      NaN
452      NaN
453      NaN
454      NaN
455      NaN
Name: item_price, dtype: object

In [15]:
chp.item_price = chp.item_price.astype(float)
# Now convert it to a floating point number.

print("Variable dtypes:\n", chp.dtypes, sep='')


Variable dtypes:
order_store_id          int32
quantity                int64
item_name              object
choice_description     object
item_price            float64
dtype: object


**Comment.** Unlike the string methods we described earlier, this use of replace affects complete entries, not elements of string entries. For example, suppose we tried to replace the periods in decimal numbers with an asterisk. We could try the following, but it doesn't work: the decimal numbers don't change.

#### Issue #4: Missing values are marked erratically.

It's important to label missing values, so that Pandas doesn't interpret entries as strings. Pandas is also smart enough to ignore things labeled missing when it does calculations or graphs. If we compute, for example, the mean of a variable, the default is to ignore missing values.

We've seen that we can label certain entries as missing values in read statements:  read_csv, read_excel, and so on. Moreover, in the operations above, we showed how to take entries that were hard to make sense of and called them missing values useing the `replace` command and `np.nan`.

**Working with missing values** Here are some operations we can do...

In [16]:
chp.order_store_id[chp.item_price.isnull()]
# These are the order numbers with null values



448    195
449    195
450    195
451    195
452    195
453    195
454    195
455    195
Name: order_store_id, dtype: int32

The next command of use is `.dropna` The one thing to note is that Pandas (when it computes things or plots) automatically drops stuff. So here is an example, the mean with the NaNs there and the mean without. They are the same.

In [17]:
print(chp.item_price.dropna().mean())
print(chp.item_price.mean())

7.454735772357705
7.454735772357705


### Some Quick Analysis

Now that we have our data set clean, lets just do acouple of things to check it out.

#### Average Order

Here lets groupby our oder number (that we worked so hard to clean), then compute the number of items and average cost...

In [19]:
quant = chp.groupby("order_store_id").quantity.sum()

cost = chp.groupby("order_store_id").item_price.sum()

print("Average Oder Size", round(quant.mean(),2), "Items")

print("Average Cost of Order", round(cost.median(),2), "Dollars")

Average Oder Size 2.53 Items
Average Cost of Order 15.7 Dollars
